In [1]:
# Permet de nettoyer notre dataset en éliminant ou en remplaçant les données manquantes 

# Simple Imputer

In [14]:
from sklearn.impute import SimpleImputer

# Il faut commencer par indiquer quelles sont les valeurs manquantes de notre dataset 

## Généralement elles sont indiquées par 'np.nan' comme dans l'exemple ci-dessous

X = np.array([[10, 3],
              [0, 4],
              [5, 3],
             [np.nan, 3]])
### Mais il peut arriver d'avoir -99, +l'infini


In [11]:
# Notre travail est justement d'identifier quelles valeurs manquantes on cherche à noettoyer

## On peut alors créer plusieurs séries de SimpleImputer piur filtrer plusieurs types de variables

### Il faut ensuite définir la 'strategy' pour remplacer ces valeurs, il en existe 4 majeures :
    
    # Moyenne
    # Mediane
    # Plusfréquente 'most
    # Une constante 
    
imputer = SimpleImputer(missing_values=np.nan,
                       strategy='mean')

imputer.fit_transform(X)

array([[10.,  3.],
       [ 0.,  4.],
       [ 5.,  3.],
       [ 5.,  3.]])

image = plt.imread('Simple_Imputer.png')
plt.figure(figsize=(9,5))
plt.imshow(image)

# KNNImputer

In [19]:
from sklearn.impute import KNNImputer

# Il remplace les valeures manquantes à un certain échantillon par les valeurs qui correspondent aux autres observations les plus 
# similaires, par les valeurs des plus proches voisins 

## C'est donc un algo de KNearestNeighbors classique dans lequel on va donc choisir un nombre de voisins à considérer

X = np.array([[1, 100],
             [2, 30],
             [3, 15],
             [np.nan, 20]])

In [16]:
imputer = KNNImputer(n_neighbors=1)
imputer.fit_transform(X)

# On voit que la valeur la plus proche de 20 est 15, et que la valeurs associée est 2, c'est donc notre résultat

array([[  1., 100.],
       [  2.,  30.],
       [  3.,  15.],
       [  3.,  20.]])

In [ ]:
# On peut alors penser à utliser GridSearchCV pour trouver le nombre de voisin otpimal ~ permettant le meilleur remplacement des NaN

# MissingIndicator

In [17]:
from sklearn.impute import MissingIndicator

# Indique où est-ce qu'il manque des données dans notre dataset


In [20]:
MissingIndicator().fit_transform(X)

array([[False],
       [False],
       [False],
       [ True]])

In [24]:
# Cela peut être très utile de créer une colonne dans notre dataset qui indiquerait si oui ou non il nous manque plusieurs 
# informations pour un certain échantillons

## Dans le cas du Titanic, un passager qui n'a pas payé et qui n'a pas de classe est surement en réalité un m'embre d'équipage ! 

### Pour cela on utilise la fonction make_union qui permet de traiter de façon différente plusieurs données dans un même dataset,
### càd de façon parallèle, puis de concaténer nos résultats dans un seul et même tableau 

from sklearn.pipeline import make_union

X = np.array([[1, 100],
             [2, 30],
             [3, 15],
             [np.nan, np.nan]])

In [25]:
# On commence donc par créer une Pipeline 

pipeline = make_union(SimpleImputer(strategy='constant', fill_value=-99),
                      MissingIndicator())

pipeline.fit_transform(X)

# On obtient alors d'un cotés (droit), les colonnes qui ont subit SimpleImputer, donc on garde nos informations sur la classe du 
# du passager de le prix de son ticket

## De l'autre une indication sur le fait que cela soit plutot un passager ou un matelos 

array([[  1., 100.,   0.,   0.],
       [  2.,  30.,   0.,   0.],
       [  3.,  15.,   0.,   0.],
       [-99., -99.,   1.,   1.]])

# Conclusion : Le manque d'information EST une information !!

# Application 

In [32]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import SGDClassifier
import seaborn as sns


In [33]:
titanic = sns.load_dataset('titanic')
X = titanic[['pclass', 'age']]
y = titanic['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [34]:
model = make_pipeline(KNNImputer(), SGDClassifier())

# Le fait d'utiliser le KNNImputer  plutôt que d'utiliser une fonction Pandas comme .fillna() pour faire de l'imputation, va nous
# permettre d'utiliser notre 'transformer' avec GrisSearchCv afin d'en optimiser ses caractéristiques


In [35]:
# On commence donc par créer un dictonnaire, dans lequel on va mettre les paramètres que l'on souhaite optimiser 

params = { 
'knnimputer__n_neighbors': [1, 2, 3, 4]
}

In [36]:
# A partir de la on peut créer une GridSearchCV

grid = GridSearchCV(model, param_grid = params, cv = 5)


In [38]:
grid.best_estimator_

# On a donc pu optimiser le nombre de paramètres de notre KNNImputer grâce à GridSearchCV, ce qui n'aurait pas été possible avec 
# des fonctions Pandas

Pipeline(steps=[('knnimputer', KNNImputer(n_neighbors=1)),
                ('sgdclassifier', SGDClassifier())])